In [14]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [28]:
hazard_rates = pd.read_csv('hazard_rates.csv')
hazard_rates.sort_values(by = 'SettlementDate', inplace=True)
hazard_rates.reset_index(inplace=True, drop=True)

discount_factors = pd.read_csv('discount_factor_common_dates.csv')
discount_factors = discount_factors.set_index('Date')
dfc = discount_factors.columns
discount_factors['zero'] = 1
discount_factors = discount_factors[np.append('zero', dfc )]
discount_factors.sort_values(by = 'Date', inplace=True)
discount_factors.reset_index(inplace=True)

coupons = pd.read_excel('treasury_bond_note_coupons.xlsx')

cds = pd.read_csv('cds_data_processed.csv')
cds.sort_values(by = 'Settlement Date', inplace=True)
cds.reset_index(inplace=True, drop=True)


In [30]:
mean_coupons = coupons.groupby('Tenor').mean().reset_index()

In [31]:
cds['Open_6m'] = cds['Open_6m']/100
cds['Open_1yr'] = cds['Open_1yr']/100
cds['Open_2yr'] = mean_coupons[mean_coupons['Tenor'] == 2]['Cpn'].values[0] - cds['Open_2yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 3]['Cpn'].values[0] - cds['Open_3yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 5]['Cpn'].values[0] - cds['Open_5yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 7]['Cpn'].values[0] - cds['Open_7yr']/100
cds['Open_3yr'] = mean_coupons[mean_coupons['Tenor'] == 10]['Cpn'].values[0] - cds['Open_10yr']/100

In [32]:
time_periods_m = np.array([0, 1/12,3/12, 6/12, 9/12, 1])
time_periods_y = np.arange(2, 31, 1)
time_periods = np.append(time_periods_m, time_periods_y)

In [33]:
discount_factors_funcs = discount_factors.apply(lambda row: interp1d(time_periods, row[1:], kind='cubic'), axis = 1)

In [34]:
cds_time_periods = [0.5, 1, 2, 3, 5, 7, 10]

In [35]:
discount_factors_subset_times = discount_factors.iloc[:, [4, 6, 7, 8,10,12,15]]
cds_subset_times = cds.iloc[:, 8:]

In [36]:
df_integrate = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate[i, j]  = integrate.quad(lambda x: discount_factors_funcs[i](x), 0,val)[0]

In [37]:
hz_moded = hazard_rates.copy()
hz_moded
hz_moded['Maturity_6m'] = np.exp(-hz_moded['Maturity_6m']*0.5)
hz_moded['Maturity_1yr'] = np.exp(-hz_moded['Maturity_1yr']*1)
hz_moded['Maturity_2yr'] = np.exp(-hz_moded['Maturity_1yr']*2)
hz_moded['Maturity_3yr'] = np.exp(-hz_moded['Maturity_1yr']*3)
hz_moded['Maturity_5yr'] = np.exp(-hz_moded['Maturity_1yr']*5)
hz_moded['Maturity_7yr'] = np.exp(-hz_moded['Maturity_1yr']*7)
hz_moded['Maturity_10yr'] = np.exp(-hz_moded['Maturity_1yr']*10)


In [38]:
hz_moded_subset_times = hz_moded.iloc[:, 1:]

In [39]:
pt_2 = np.multiply(np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times), hz_moded_subset_times)

/var/folders/34/gh568vr946d45y78dns07shm0000gn/T/ipykernel_49356/4021414376.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  pt_2 = np.multiply(np.add(np.multiply(cds_subset_times, df_integrate), discount_factors_subset_times), hz_moded_subset_times)


In [ ]:
integrate.quad(lambda x: discount_factors_funcs[i](x), 0,val)[0]

In [41]:
integrate.quad(lambda t: integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0], 0, val)[0]

46.555670494467975

In [46]:
hz_moded_subset_times

,Maturity_6m,Maturity_1yr,Maturity_2yr,Maturity_3yr,Maturity_5yr,Maturity_7yr,Maturity_10yr
0,0.997844,0.995693,0.136506,0.050435,0.006885,0.000940,0.000047
1,0.998046,0.996095,0.136396,0.050374,0.006871,0.000937,0.000047
2,0.997952,0.995876,0.136456,0.050407,0.006878,0.000939,0.000047
3,0.999421,0.998842,0.135649,0.049960,0.006777,0.000919,0.000046
4,0.998551,0.996949,0.136164,0.050245,0.006842,0.000932,0.000047
...,...,...,...,...,...,...,...
3154,0.998657,0.997425,0.136034,0.050173,0.006825,0.000928,0.000047
3155,0.999384,0.998285,0.135800,0.050044,0.006796,0.000923,0.000046
3156,0.999352,0.998256,0.135808,0.050048,0.006797,0.000923,0.000046
3157,0.998980,0.997721,0.135954,0.050129,0.006815,0.000927,0.000046


In [ ]:
df_integrate_double = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate_double[i, j]  = integrate.quad(lambda t: hz_moded_subset_times[i,j] * np.exp(-hz_moded_subset_times[i,j]*t) * integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0] + discount_factors_funcs[i](t), 0, val)[0]

In [53]:
df_integrate_double = np.empty(shape =(cds.shape[0],len(cds_time_periods )))
for i in range(discount_factors.shape[0]):
    for j, val in enumerate(cds_time_periods):
        df_integrate_double[i,j] = integrate.quad(lambda t: hz_moded_subset_times.iloc[i,j] * np.exp(-hz_moded_subset_times.iloc[i,j]*t) * cds_subset_times.iloc[i,j]* integrate.quad(lambda x: discount_factors_funcs[i](x), 0,t)[0] + discount_factors_funcs[i](t), 0, val)[0]

array([[2.61911124e-314, 2.61911126e-314, 2.61911128e-314, ...,
        2.61911131e-314, 2.61911132e-314, 2.61911134e-314],
       [2.61911136e-314, 2.61911137e-314, 2.61911139e-314, ...,
        2.61911142e-314, 2.61911143e-314, 2.61911145e-314],
       [2.61911147e-314, 2.61911148e-314, 2.61911150e-314, ...,
        2.61911153e-314, 2.61911155e-314, 2.61911156e-314],
       ...,
       [2.61847852e-314, 2.61847854e-314, 2.61847856e-314, ...,
        2.61847859e-314, 2.61847860e-314, 2.61847862e-314],
       [2.61847864e-314, 2.61847865e-314, 2.61847867e-314, ...,
        2.61847870e-314, 2.61847871e-314, 2.61847873e-314],
       [2.61847875e-314, 2.61847876e-314, 2.61847878e-314, ...,
        2.61847881e-314, 2.61847882e-314, 2.61847884e-314]])